### Import der wichtigsten Bibliotheken für Modelltraining und Monitoring 
In diesem Abschnitt werden PyTorch-Module für neuronale Netzwerke und Datenverarbeitung, Hugging Face-Tools für Tokenisierung und das Laden von Datensätzen sowie Weights & Biases (wandb) für das Tracking und die Visualisierung von Experimenten importiert.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import GPT2Tokenizer
from datasets import load_dataset
import wandb


### Laden des Wikitext-2-Datensatzes
Hier wird der Wikitext-2-raw-v1-Datensatz mit Hugging Face's load_dataset-Funktion geladen, um ihn für das Training oder die Evaluierung eines Modells vorzubereiten.


In [ ]:
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")


### Initialisierung und Anpassung des GPT-2-Tokenizers 
Der vortrainierte GPT-2-Tokenizer wird geladen und das Padding-Token wird auf das End-of-Sequence-Token (eos_token) gesetzt, um die Konsistenz bei der Textverarbeitung sicherzustellen.

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Ensure EOS is used as pad


### Tokenisierung und Vorbereitung der Trainings- und Validierungsdaten
Eine Tokenisierungsfunktion wird definiert, die Texte auf eine maximale Länge von 256 Tokens zuschneidet und auffüllt. Anschließend werden der Trainings- und Validierungsdatensatz mithilfe dieser Funktion verarbeitet und die ursprüngliche Textspalte entfernt.



In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)

train_dataset = dataset["train"].map(tokenize_function, batched=True, remove_columns=["text"])
val_dataset = dataset["validation"].map(tokenize_function, batched=True, remove_columns=["text"])


### Formatierung der Datensätze für PyTorch
Die Trainings- und Validierungsdatensätze werden so formatiert, dass nur die input_ids im PyTorch-Format ausgegeben werden – ideal für das direkte Training mit Modellen.

In [ ]:
train_dataset.set_format(type="torch", columns=["input_ids"])
val_dataset.set_format(type="torch", columns=["input_ids"])


### Definition der Positional Encoding Klasse
Diese Klasse implementiert Positional Encoding, um Positionsinformationen durch Sinus- und Kosinusfunktionen zu den Eingabe-Embeddings hinzuzufügen – ein essenzieller Bestandteil bei der Verarbeitung von Sequenzdaten in Transformermodellen.

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()
        self.encoding = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1).float()
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(torch.log(torch.tensor(10000.0)) / d_model))
        self.encoding[:, 0::2] = torch.sin(position * div_term)
        self.encoding[:, 1::2] = torch.cos(position * div_term)
        self.encoding = self.encoding.unsqueeze(0)

    def forward(self, x):
        return x + self.encoding[:, :x.size(1)].to(x.device)


### Implementierung eines einfachen Transformer-Decodermodells
Diese Klasse definiert ein Transformer-basiertes Decoder-Modell mit Embedding-, Positionskodierungs- und Decoder-Schichten. Sie verarbeitet Eingabesequenzen autoregressiv und projiziert die Ausgaben zurück auf den Wortschatzraum für Aufgaben wie Sprachmodellierung.



In [ ]:
class SimpleTransformerDecoderModel(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4, num_layers=2, max_seq_len=256):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = PositionalEncoding(d_model, max_len=max_seq_len)
        
        decoder_layer = nn.TransformerDecoderLayer(d_model=d_model, nhead=nhead)
        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        
        self.output_layer = nn.Linear(d_model, vocab_size)
        self.d_model = d_model

    def generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.full((sz, sz), float('-inf')), diagonal=1)

    def forward(self, tgt_ids):
        """
        tgt_ids: [batch_size, seq_len] - token ids
        """
        device = tgt_ids.device
        x = self.embedding(tgt_ids) * (self.d_model ** 0.5)  # scale embeddings
        x = self.pos_encoding(x).transpose(0, 1)  # [seq_len, batch_size, d_model]

        # Causal mask
        seq_len = x.size(0)
        tgt_mask = self.generate_square_subsequent_mask(seq_len).to(device)

        # Fake memory — just pass zeros to satisfy TransformerDecoder API
        memory = torch.zeros_like(x)

        output = self.transformer_decoder(tgt=x, memory=memory, tgt_mask=tgt_mask)
        output = output.transpose(0, 1)  # [batch_size, seq_len, d_model]
        return self.output_layer(output)


### Trainingsschleife für das Transformer-Decodermodell
Diese Funktion führt das Training des Modells durch, indem sie Eingabesequenzen vorbereitet, Vorhersagen erzeugt, den Verlust berechnet, Gradienten zurückpropagiert und die Modellparameter optimiert.



In [ ]:
def train(model, dataloader, optimizer, criterion):
    model.train()
    total_loss = 0
    for batch in dataloader:
        input_ids = batch["input_ids"]
        inputs = input_ids[:, :-1]
        targets = input_ids[:, 1:]

        optimizer.zero_grad()
        output = model(inputs)  # Pass only inputs
        loss = criterion(output.view(-1, output.size(-1)), targets.reshape(-1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(dataloader)


### Evaluierungsfunktion für das Transformer-Decodermodell
Diese Funktion bewertet das Modell auf Validierungsdaten, indem sie den Verlust ohne Gradientenberechnung ermittelt, um die Trainingsqualität objektiv zu überprüfen.

In [ ]:
def evaluate(model, dataloader, criterion):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch["input_ids"]
            inputs = input_ids[:, :-1]
            targets = input_ids[:, 1:]

            output = model(inputs)  # Only inputs
            loss = criterion(output.view(-1, output.size(-1)), targets.reshape(-1))
            total_loss += loss.item()
    return total_loss / len(dataloader)


### Initialisierung von Weights & Biases für Experiment-Tracking
Mit wandb.login() wird die Verbindung zu Weights & Biases hergestellt. Danach wird die Konfiguration für das Experiment festgelegt, einschließlich Hyperparametern und Modellarchitektur. wandb.init() startet das Tracking des Trainingsprozesses unter dem Projekt "language-model".

In [ ]:
wandb.login()  # Only required once per session/machine

config = {
    "epochs": 5,
    "batch_size": 4,
    "learning_rate": 5e-4,
    "architecture": "TransformerDecoder",
    "dataset": "WikiText-2",
    "vocab_size": len(tokenizer),
    "embedding_dim": 128,
    "nhead": 4,
    "num_layers": 2,
    "max_seq_len": 256
}

wandb.init(project="language-model", config=config)


### Modellinitialisierung und Setup von Optimizer und Dataloader
Ein SimpleTransformerDecoderModel wird mit den aus Weights & Biases (wandb.config) entnommenen Hyperparametern erstellt. Der Adam-Optimizer und die Kreuzentropie-Verlustfunktion werden festgelegt. Zudem werden Dataloader für Training und Validierung vorbereitet, um die Daten in Batches zu laden.

In [ ]:
model = SimpleTransformerDecoderModel(
    vocab_size=wandb.config.vocab_size,
    d_model=wandb.config.embedding_dim,
    nhead=wandb.config.nhead,
    num_layers=wandb.config.num_layers,
    max_seq_len=wandb.config.max_seq_len
)

optimizer = torch.optim.Adam(model.parameters(), lr=wandb.config.learning_rate)
criterion = nn.CrossEntropyLoss()

train_loader = DataLoader(train_dataset, batch_size=wandb.config.batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=wandb.config.batch_size)


### Training und Evaluierung über Epochen mit Weights & Biases Logging
Für jede Epoche wird das Modell trainiert und evaluiert. Der Trainings- und Validierungsverlust wird berechnet und ausgedruckt. Diese Werte werden dann zu Weights & Biases geloggt, um den Fortschritt des Experiments zu überwachen.

In [ ]:
for epoch in range(wandb.config.epochs):
    print(f"Epoch {epoch + 1}")
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss = evaluate(model, val_loader, criterion)

    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "val_loss": val_loss
    })


### Speichern des Modells und Tokenizers
Das Modell wird in einem angegebenen Verzeichnis gespeichert, indem die Gewichtungen mit torch.save gesichert werden. Zudem wird der Tokenizer im gleichen Verzeichnis mit save_pretrained abgelegt, um das Modell später wieder zu laden.

In [ ]:
import os
import torch

# Create directory if it doesn't exist
model_save_path = "./simple_transformer_model"
os.makedirs(model_save_path, exist_ok=True)

# Save model weights
torch.save(model.state_dict(), model_save_path + "/pytorch_model.bin")

# Save tokenizer
tokenizer.save_pretrained(model_save_path)


### Laden des gespeicherten Modells und Setzen auf Evaluierungsmodus
Das Modell wird mit der gleichen Architektur neu erstellt und die gespeicherten Gewichtungen werden mit load_state_dict geladen. Anschließend wird das Modell in den Evaluierungsmodus versetzt, um Vorhersagen zu treffen.



In [ ]:
# Rebuild the model class
model = SimpleTransformerDecoderModel(
    vocab_size=len(tokenizer),
    d_model=128,
    nhead=4,
    num_layers=2,
    max_seq_len=256
)

# Load weights
model.load_state_dict(torch.load(model_save_path + "/pytorch_model.bin"))
model.eval()


### Textgenerierung mit dem Transformer-Modell
Diese Funktion verwendet das Modell, um Text basierend auf einem Eingabe-Prompt zu generieren. Sie tokenisiert den Prompt, verwendet das Modell zur Vorhersage der nächsten Tokens und dekodiert schließlich die generierten Tokens zurück in lesbaren Text.



In [ ]:
def generate_text(model, tokenizer, prompt, max_length=100):
    model.eval()
    # Tokenize the prompt text
    input_ids = tokenizer.encode(prompt, return_tensors='pt')

    # Generate text (we only use `tgt` part, no memory required for decoding)
    with torch.no_grad():
        for _ in range(max_length):
            output = model(input_ids)
            next_token_logits = output[:, -1, :]
            
            # Sample from the distribution of possible next tokens
            next_token_id = torch.argmax(next_token_logits, dim=-1).unsqueeze(-1)
            input_ids = torch.cat((input_ids, next_token_id), dim=-1)
    
    # Decode the generated tokens
    generated_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    return generated_text

# Example usage:
prompt = "Once upon a time"
generated_text = generate_text(model, tokenizer, prompt)
print(generated_text)


### Authentifizierung bei Hugging Face Hub
Mit der login()-Funktion von Hugging Face wird der Benutzer mit einem API-Token authentifiziert, um auf Modelle und Datasets vom Hugging Face Hub zuzugreifen.